In [1]:
import pandas as pd

In [2]:
pd.__version__

'2.2.1'

In [3]:
!pip install pyarrow

In [4]:
df_january_2023 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [5]:
df_february_2023 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [6]:
df_january_2023

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


Answer: 19

## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [7]:
df_january_2023.tpep_pickup_datetime = pd.to_datetime(df_january_2023.tpep_pickup_datetime)
df_january_2023.tpep_dropoff_datetime = pd.to_datetime(df_january_2023.tpep_dropoff_datetime)

In [8]:
df_january_2023['duration'] = df_january_2023.tpep_dropoff_datetime - df_january_2023.tpep_pickup_datetime 
df_january_2023['duration'] = df_january_2023['duration'].apply(lambda td: td.total_seconds() / 60)

In [9]:
df_january_2023['duration']

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [10]:
df_january_2023['duration'].std()

42.59435124195458

Answer: 42.59

## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?


In [11]:
df_january_2023_filtered = df_january_2023[(df_january_2023.duration >=1) & (df_january_2023.duration <= 60)]

In [12]:
len(df_january_2023_filtered)*100 / len(df_january_2023)

98.1220282212598

Answer: 98%

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [13]:
categorical = ['PULocationID', 'DOLocationID']

df_january_2023_filtered[categorical] = df_january_2023_filtered[categorical].astype(str)

/tmp/ipykernel_29777/234229622.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_january_2023_filtered[categorical] = df_january_2023_filtered[categorical].astype(str)


In [14]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()

train_dicts = df_january_2023_filtered[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)


In [15]:
X_train.shape

(3009173, 515)

Answer: 515

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


target = 'duration'
y_train = df_january_2023_filtered[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [17]:
mean_squared_error(y_train, y_pred, squared=False)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649261932106969

Answer: 7.64

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [18]:
df_february_2023['duration'] = df_february_2023.tpep_dropoff_datetime - df_february_2023.tpep_pickup_datetime 
df_february_2023['duration'] = df_february_2023['duration'].apply(lambda td: td.total_seconds() / 60)
df_february_2023_filtered = df_february_2023[(df_february_2023.duration >=1) & (df_february_2023.duration <= 60)]
df_february_2023_filtered[categorical] = df_february_2023_filtered[categorical].astype(str)

/tmp/ipykernel_29777/1686025734.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_february_2023_filtered[categorical] = df_february_2023_filtered[categorical].astype(str)


In [19]:
val_dicts = df_february_2023_filtered[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

X_val = dv.transform(val_dicts)
y_val = df_february_2023_filtered[target].values


In [20]:
X_val.shape

(2855951, 515)

In [21]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811818743246608

Answer: 7.81